<div class="alert alert-block alert-info" style="background-color: #05299E; color: white;">
    <h1 style="color: white;">LangGraph Agent Client</h1>
    <h4 style="color: white;">LangGraph agents connecting to MCP server of OpenAI agents</h4>
</div>

In [ ]:
#pip install langchain-mcp-adapters

In [17]:
import asyncio
import nest_asyncio
import os
from dotenv import load_dotenv
import sys

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
from typing import List, Dict, Any, Tuple, Optional

In [18]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [19]:
nest_asyncio.apply()

In [20]:
# Define our agent state
class AgentState(Dict[str, Any]):
    messages: List[Dict[str, str]]
    tool_responses: List[Dict[str, str]] = []

In [21]:
async def run_langgraph_agent():
    """Run a LangGraph agent that connects to the MCP server"""
    
    # Initialize the LLM
    llm = ChatOpenAI(model="gpt-4o-mini")
    
    print("Connecting to MCP server...")
    
    # Connect to the MCP server
    async with MultiServerMCPClient(
        {
            "MCP server": {
                "url": "http://localhost:8000/sse",
                "transport": "sse",
            }
        }
    ) as client:
        # Get available tools from the MCP server
        tools = client.get_tools()
        print(f"Connected successfully. Available tools: {[t.name for t in tools]}")
        
        # Create a LangGraph ReAct agent with the MCP tools
        agent = create_react_agent(llm, tools)
        
        # Now try with the graph
        print("\nTesting graph invocation:")
        try:
            # Create a simple workflow graph
            workflow = StateGraph(AgentState)
            
            # Define agent node that properly handles async
            async def agent_node_async(state):
                return await agent.ainvoke({"messages": state["messages"]})
                
            # Wrapper for the agent node
            def agent_node(state: AgentState) -> Dict[str, Any]:
                loop = asyncio.get_event_loop()
                result = loop.run_until_complete(agent_node_async(state))
                return {"messages": result["messages"]}
            
            # Add the agent node to the graph
            workflow.add_node("agent", agent_node)
            
            # Define the graph edges
            workflow.add_edge(START, "agent")
            workflow.add_edge("agent", END)
            
            # Compile the graph
            app = workflow.compile()
            
            # Execute the graph with a user query
            result = app.invoke({
                "messages": [
                    {"role": "user", "content": "what is eldershield used for?"}
                ]
            })
            
            # Print the result
            print("\nGraph Invocation Result:")
            print(result)
            return result
                    
        except Exception as e:
            print(f"Error with graph execution: {e}")

In [22]:
print("Starting test...")
x = await run_langgraph_agent()

Starting test...
Connecting to MCP server...
Connected successfully. Available tools: ['healthcare_finance_domain']

Testing graph invocation:

Graph Invocation Result:
{'messages': [HumanMessage(content='what is eldershield used for?', additional_kwargs={}, response_metadata={}, id='ab47cdef-31ba-4574-8a3b-d199c3d11d01'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_V1RpveuZhHrKf3WgUpXJ16LL', 'function': {'arguments': '{"query":"What is Eldershield used for?"}', 'name': 'healthcare_finance_domain'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 83, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'finish_reason': 'tool_calls', 'logprobs': 

In [23]:
x['messages'][-1].content

'ElderShield is a long-term care insurance scheme in Singapore designed to provide financial support for individuals who become severely disabled and require long-term care. It offers cash payouts to help cover the costs of care needed for daily activities, such as getting dressed, eating, or moving around.\n\nIf you have specific questions about eligibility, premiums, or application procedures for ElderShield, feel free to ask!'

In [24]:
x['messages']

[HumanMessage(content='what is eldershield used for?', additional_kwargs={}, response_metadata={}, id='ab47cdef-31ba-4574-8a3b-d199c3d11d01'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_V1RpveuZhHrKf3WgUpXJ16LL', 'function': {'arguments': '{"query":"What is Eldershield used for?"}', 'name': 'healthcare_finance_domain'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 83, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f73cd577-3ee8-429b-9466-ab97046664a7-0', tool_calls=[{'name': 'healthcare_finance_domain', 'args': {'query': 'What is Eldershield used for?'}, 'id': 'call_V1RpveuZhHr

In [ ]:
#first is user query
#second, no reply from AI, invoked the healthcare_finance_domain tool in MCP server
#third, the tool (healthcare_finance_domain = agent network) returned the result 
#fourth, using the result (response from agent network) from the tool, the LG agent gave its reply (which is very similar and slightly different)

content='what is eldershield used for?' additional_kwargs={} response_metadata={} id='ab47cdef-31ba-4574-8a3b-d199c3d11d01'
content='' additional_kwargs={'tool_calls': [{'id': 'call_V1RpveuZhHrKf3WgUpXJ16LL', 'function': {'arguments': '{"query":"What is Eldershield used for?"}', 'name': 'healthcare_finance_domain'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 83, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-f73cd577-3ee8-429b-9466-ab97046664a7-0' tool_calls=[{'name': 'healthcare_finance_domain', 'args': {'query': 'What is Eldershield used for?'}, 'id': 'call_V1RpveuZhHrKf3WgUpXJ16LL', 'type': 'tool_call